In [40]:
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn
from collections import Counter
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm
import torch
from torch.nn import CrossEntropyLoss
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from torch.nn.functional import softmax
import numpy as np
from tqdm import tqdm

In [2]:
with open('../Data/rft_clauses_final.json') as f:
    data = json.load(f)

In [3]:
len(data)

510

In [4]:
def transform_data_to_lists(data):
    content_list = []
    clause_type_list = []
    for doc_id, doc in data.items():
        for clause_type, clauses in doc["clauses"].items():
            for clause in clauses:
                content_list.append(clause)
                clause_type_list.append(clause_type)
    return content_list, clause_type_list
content_list, clause_type_list = transform_data_to_lists(data)

In [5]:
print(len(content_list))
print(len(clause_type_list))

13101
13101


In [6]:
content_list[15]

'INITIAL ORDER COMMITMENT - MA commits to purchase a minimum of 100 Units in aggregate within the Territory within the first six months of term of this Agreement.'

In [7]:
clause_type_list[15]

'MINCOM'

In [9]:
clause_type_list[0]

'DOC_NAME'

In [10]:
content_list[0]

'MARKETING AFFILIATE AGREEMENT'

## BILOC data

In [12]:
with open('../Data/biloc_tagged_clauses.json') as f:
    d = json.load(f)

In [13]:
data_files ='../Data/biloc_tagged_clauses.json'
datasets = load_dataset('json', data_files=data_files, field='data')
test_size=0.15
random_seed=42

datasets = datasets['train'].train_test_split(test_size=test_size, seed=random_seed)
print(datasets)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['split_tokens', 'id', 'ner_tags'],
        num_rows: 401
    })
    test: Dataset({
        features: ['split_tokens', 'id', 'ner_tags'],
        num_rows: 71
    })
})


In [14]:

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
def tokenize_and_align_labels(tokenizer, examples):
    tokenized_inputs = tokenizer(examples["split_tokens"], truncation=True, padding="max_length", is_split_into_words=True, return_tensors="pt")
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
def convert_to_tensors(batch):
    batch_tensors = {key: tensor(value) for key, value in batch.items()}

In [16]:
tokenized_datasets = datasets.map(lambda examples: tokenize_and_align_labels(tokenizer, examples), batched=True)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

In [42]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)


In [43]:
model_name = "bert-base-cased"  
num_labels = 165

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:


optimizer = AdamW(model.parameters(), lr=5e-5)
loss_function = CrossEntropyLoss(ignore_index=-100)  # Assuming -100 is used to ignore padding or special tokens

num_epochs = 30 # Adjust as necessary
device = torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))
model.to(device)

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    # Wrap train_loader with tqdm for a progress bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()  # Reset gradients

        outputs = model(**batch)
        logits = outputs.logits  # Correctly access logits
        loss = loss_function(logits.view(-1, num_labels), batch['labels'].view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Optionally, you can update the progress bar with the current loss
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

Epoch 1/30: 100%|██████████| 201/201 [00:18<00:00, 10.65it/s, loss=0.3223]


Epoch 1/30, Average Loss: 0.8366


Epoch 2/30: 100%|██████████| 201/201 [00:18<00:00, 10.64it/s, loss=0.1642]


Epoch 2/30, Average Loss: 0.4722


Epoch 3/30: 100%|██████████| 201/201 [00:18<00:00, 10.63it/s, loss=0.1311]


...

Epoch 28/30, Average Loss: 0.0245


Epoch 29/30: 100%|██████████| 201/201 [00:40<00:00,  4.94it/s, loss=0.0149]


Epoch 29/30, Average Loss: 0.0216


Epoch 30/30: 100%|██████████| 201/201 [00:40<00:00,  4.94it/s, loss=0.0048]

Epoch 30/30, Average Loss: 0.0160


In [45]:
true_labels = []
pred_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        
        
        true_labels.extend(batch["labels"].cpu().numpy())
        pred_labels.extend(predictions.cpu().numpy())


true_labels_flat = [label for sublist in true_labels for label in sublist]
pred_labels_flat = [pred for sublist in pred_labels for pred in sublist]

precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, pred_labels_flat, average='weighted', labels=np.unique(pred_labels_flat))
accuracy = accuracy_score(true_labels_flat, pred_labels_flat) 

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')

Precision: 0.8925
Recall: 0.9117
F1: 0.8999
Accuracy: 0.8879


/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = total_params - trainable_params

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")
print(f"Non-trainable Parameters: {non_trainable_params}")


Total Parameters: 107846565
Trainable Parameters: 107846565
Non-trainable Parameters: 0
